In [1]:
# 배치 추가한거 테스트

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml
from layers import MultiLayer
from layers import Relu, Affine, SoftmaxWithLoss, Sigmoid
from collections import OrderedDict

In [3]:
def make_one(x):
    if x.ndim == 1: 
        t = np.zeros((x.size,np.unique(x).size))
        for i in range(t.shape[0]):
            t[i,x[i]]=1
        return t

In [4]:
#테스트1

In [5]:
from sklearn.datasets import load_iris
X = load_iris()['data']
y = load_iris()['target']
y = make_one(y)

In [6]:
class SGD:
    def __init__(self,lr=1e-3):
        self.lr = lr
    
    def minimize(self, w, grads):
        for key in w.keys():
            w[key] -=  self.lr*grads[key]        

In [7]:
optimizer = SGD(lr=1e2)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.4)
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test, test_size=.5)

In [9]:
input_size = X.shape[1]
hidden_size = [5,10]
output_size = y.shape[1]
model = MultiLayer(input_size,hidden_size,output_size)

In [10]:
epochs = 10
lr = 1e-3
model.fit_gd(epochs,lr,X_train,y_train,X_val,y_val)

epoch  0 :val_loss=========== 3.2870424046850295 val_acc:======== 0.4


In [11]:
epochs = 10
lr = 1e-3
batch_size = 100
model.fit_sgd(batch_size,epochs,lr,X_train,y_train,X_val,y_val)

epoch  0 :val_loss=========== 1.9805677226513587 val_acc:======== 0.5
epoch  20 :val_loss=========== 1.141057714117284 val_acc:======== 0.5
epoch  40 :val_loss=========== 1.141057714117284 val_acc:======== 0.5
epoch  60 :val_loss=========== 1.141057714117284 val_acc:======== 0.5
epoch  80 :val_loss=========== 1.141057714117284 val_acc:======== 0.5


C:\Users\user\Desktop\수업노트\이강욱강사\9.DeepLearning\loss.py:11: RuntimeWarning: invalid value encountered in double_scalars
  return -np.sum(np.log(y+delta) * t)/y.shape[0]


In [12]:
epochs = 100
for epoch in range(epochs):
    grads = model.gradient(X_train,y_train)
    optimizer.minimize(model.W,grads)
    if epoch % 20 == 0:
        print("epoch ",epoch,":val_loss===========",model.loss(X_test,y_test),"val_acc:========",model.accuracy(X_test,y_test))
        model.loss_val.append(model.loss(X_test,y_test))
        model.acc_val.append(np.round(model.accuracy(X_test,y_test),2))

epoch  0 :val_loss=========== 11.819936784036104 val_acc:======== 0.26666666666666666
epoch  20 :val_loss=========== 11.817767939231464 val_acc:======== 0.26666666666666666
epoch  40 :val_loss=========== 5.379369831329082 val_acc:======== 0.4666666666666667
epoch  60 :val_loss=========== 8.971053554236669 val_acc:======== 0.26666666666666666
epoch  80 :val_loss=========== 11.819844056082397 val_acc:======== 0.26666666666666666


In [13]:
#테스트2

In [14]:
def make_one(x):
    if x.ndim == 1:
        t = np.zeros((x.size,np.unique(x).size))
        for i in range(t.shape[0]):
            t[i,x[i]] = 1
        return t
    
class SGD: #GD(기울기 줄어듦)다음에 나온거  
    def __init__(self,lr=1e-3):
        self.lr = lr
    
    def minimize(self, w, grads):
        for key in w.keys():
            w[key] -=  self.lr*grads[key]      

In [15]:
mnist = fetch_openml('mnist_784')
X = mnist['data']
y = mnist['target']
X = X.astype(np.float32).values/255.
y = y.astype(np.int32).values
y = make_one(y)
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

input_size = X.shape[1]
hidden_size = [100,10]
output_size = y.shape[1]
model = MultiLayer(input_size,hidden_size,output_size)

In [16]:
epochs = 10
lr = 1e-3
model.fit_gd(epochs,lr,X_train,y_train,X_val,y_val)

ValueError: shapes (30,4) and (784,100) not aligned: 4 (dim 1) != 784 (dim 0)

In [ ]:
epochs = 10
lr = 1e-3
batch_size = 100
model.fit_sgd(batch_size,epochs,lr,X_train,y_train,X_val,y_val)

In [17]:
#함수만들기
class Momentum: #ppt74참고
    def __init__(self,lr=1e-3,m=0.9):
        self.lr = lr
        self.momentum = m
        self.v = None
    
    def minimize(self,w,grads):
        if self.v is None:
            self.v = {}
            for k,v in w.items():
                self.v[k] = np.zeros_like(v)
        for key in w.keys():
            self.v[key] = self.momentum + self.v[key] - self.lr*grads[key]
            w[key] += self.v[key]

In [18]:
x = np.random.randn(3,4)
y= np.zeros_like(x)
y

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [19]:
for k,v in model.W.items():
    print(k,v)

W1 [[ 0.26389972 -0.19289069 -2.73216271 ... -0.60595153  0.47866701
   0.40225521]
 [-0.58526239 -0.24354131  0.22098327 ...  0.5312799  -0.69712539
  -1.74508419]
 [-0.10312542 -0.21634066  0.45210696 ... -1.6362799   0.45629892
  -1.38826328]
 ...
 [-0.53419543 -1.76254348  0.52323223 ...  0.30983205 -0.50372048
  -0.10554803]
 [-0.10772109 -0.72804088 -0.25444966 ... -0.6059102   0.28095409
   0.43019513]
 [-0.80019661  0.43241529  0.66525522 ...  0.05485265  0.61789913
   0.76464462]]
b1 [-2.33914     0.33510675 -1.0544355  -1.1295744   0.79738766  0.12583238
 -0.39463579 -0.4386865  -0.97309656 -0.07889688  0.92090976  0.74303138
 -0.05956671 -0.05426738  1.24281642 -0.79853498  1.73035077  0.7948795
  1.96303201 -0.08232977  0.3547347  -0.61462373 -0.30434817 -2.73973894
  2.15265032  1.12839878 -0.16498597 -0.22251861  1.35980335 -1.02454584
 -0.83444751 -0.52242984  1.5660635  -1.4508155   0.14609344  0.93353
 -1.00096567  1.07801196 -1.63630794  0.80745103 -0.03150504  0.9829

In [ ]:
#모멘텀 함수 돌려보기

In [20]:
optimizer = Momentum(lr=1e3)

In [22]:
epochs = 10
for epoch in range(epochs):
    grads = model.gradient(X_train,y_train)
    optimizer.minimize(model.W,grads)
    if epoch % 20 == 0:
        print("epoch ",epoch,":val_loss===========",model.loss(X_test,y_test),"val_acc:========",model.accuracy(X_test,y_test))
        model.loss_val.append(model.loss(X_test,y_test))
        model.acc_val.append(np.round(model.accuracy(X_test,y_test),2))

epoch  0 :val_loss=========== 14.528160634381642 val_acc:======== 0.09864285714285714


In [23]:
def make_one(x):
    if x.ndim == 1:
        t = np.zeros((x.size,np.unique(x).size))
        for i in range(t.shape[0]):
            t[i,x[i]] = 1
        return t

In [24]:
class SGD: 
    def __init__(self,lr=1e-3):
        self.lr = lr
    
    def minimize(self, w, grads):
        for key in w.keys():
            w[key] -=  self.lr*grads[key]       

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

ValueError: Found input variables with inconsistent numbers of samples: [70000, 3]

In [26]:
input_size = X.shape[1]
hidden_size = [100,10]
output_size = y.shape[1]
model = MultiLayer(input_size,hidden_size,output_size)